In [1]:
import tensorflow as tf
import numpy as np
#get cifar10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data();
print(x_train.shape)
print(y_train.shape)

C:\Users\zhaon\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(50000, 32, 32, 3)
(50000, 1)


In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import scale
onehot_encoder = OneHotEncoder(sparse=False)
y_train_ohe = onehot_encoder.fit_transform(y_train);
print(y_train_ohe.shape)
print(type(y_train_ohe))
y_test_ohe =onehot_encoder.fit_transform(y_test); 

(50000, 10)
<class 'numpy.ndarray'>


In [8]:
## resblock definition
def res_block(input_tensor, kernel_size, filters):
    '''
    resnet is simply a number of these blocks stacked togther
    #for the shortcut, we need to add in a 1x1 conv in situations
    # where input and output shapes are not equal (by shape, we mean num_channels)

    :param input
    :return:
    '''

    filters1, filters2, filters3 = filters
    shortcut = input_tensor ## preserve the input
    #1d conv so we can scale the num_channels
    x = tf.layers.conv2d(input_tensor, filters = filters1, kernel_size = (1, 1))
    x = tf.layers.batch_normalization(x)
    x = tf.nn.relu(x)

    #apply the conv
    x = tf.layers.Conv2D(filters2, kernel_size,
               padding='same')(x)
    x = tf.layers.batch_normalization(x)
    x = tf.nn.relu(x)

    #1d conv so we can project back to the original conv size
    x = tf.layers.conv2d(x, filters = filters3, kernel_size =(1, 1))
    x = tf.layers.batch_normalization(x)

    x = tf.add(x, input_tensor)
    x = tf.nn.relu(x);
    return x

def encoder_resnet(input_batch):
    conv7 = tf.layers.conv2d(inputs=input_batch, filters=32, kernel_size=(2, 2))
    # use a for loop for the remaining 5 3x3 convs
    inputs = conv7;
    kernel_size = (2,2)
    filter_array = [64,64,32]

    ## first downsampling conv layer
    inputs = tf.layers.batch_normalization(inputs);
    inputs = tf.nn.relu(inputs);

    ## define first resblock
    for i in range(2):
        inputs = res_block(inputs, kernel_size, filters = filter_array)
    kernel_size = (2,2);

    ## second downsampling conv layer
    inputs = tf.layers.conv2d(inputs, filters=32, kernel_size=(3,3))
    inputs = tf.layers.batch_normalization(inputs);
    inputs = tf.nn.relu(inputs);
    inputs = tf.layers.max_pooling2d(inputs, pool_size=(2,2), strides=(2, 2))

    ## define second resblock
    filter_array2 = [128,128, 32];
    for i in range(3):
        conv_out = res_block(inputs, kernel_size, filters = filter_array2)

    ## third downsampling conv layer
    inputs = tf.layers.conv2d(conv_out, filters=32, kernel_size=(2,2))
    inputs = tf.layers.batch_normalization(inputs);
    inputs = tf.nn.relu(inputs);
    inputs = tf.layers.max_pooling2d(inputs, pool_size=(2,2), strides=(2, 2))

    ## define second resblock
    filter_array2 = [64,64,32];
    for i in range(3):
        conv_out = res_block(inputs, kernel_size, filters = filter_array2)


    batch_norm = tf.layers.batch_normalization(conv_out)
    dropout = tf.layers.dropout(batch_norm, rate=0.4);  # rate is the drop rate
    pool3 = tf.layers.max_pooling2d(inputs=dropout, pool_size=[2, 2], strides=2)
    pool7 = pool3;
    # add in dense layer
    pool_flat = tf.contrib.layers.flatten(pool7)
    dense = tf.layers.dense(inputs=pool_flat, units=1024, activation=tf.nn.relu)

    # run this
    return dense;


In [9]:
#take a batch
batch_size = 1000
[N,H,W,C] = x_train.shape;
num_batches = int(N/batch_size);

In [10]:
#define a resnet model
# define a resnet architecture using layers api
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [None, 32, 32, 3], name = 'X')
input_labels = tf.placeholder(tf.int32, shape = [None, 10], name = 'y')
inputs = X;

final_dense = encoder_resnet(inputs);
scores = tf.layers.dense(final_dense, 10);

In [11]:
## optimize the graph
loss= tf.nn.softmax_cross_entropy_with_logits_v2(labels = input_labels, logits = scores);
loss = tf.reduce_mean(loss);
optimizer = tf.train.AdamOptimizer();
optimizer = optimizer.minimize(loss);

init = tf.global_variables_initializer()
sess = tf.Session();
sess.run(init)
epochs = 64; print(num_batches)
for i in range(epochs):
    for j in range(num_batches):
        X_batch = x_train[j*batch_size:(j+1)*batch_size, :, :, :];
        y_batch = y_train_ohe[j*batch_size:(j+1)*batch_size, :];
        y_batch_flat = y_train[j*batch_size:(j+1)*batch_size, :]        
        sess.run(optimizer, feed_dict = {X: X_batch, input_labels: y_batch})
        #if(j == num_batches-1): print(sess.run(loss, feed_dict = {X: X_batch, input_labels: y_batch}))

    if(i%10 == 0):
        print('epoch: '+str(i)+' loss: '+str(sess.run(loss, feed_dict = {X: X_batch, input_labels: y_batch})))

50
epoch: 0 loss: 1.7779921
epoch: 10 loss: 0.836677


KeyboardInterrupt: 